# Emerging Technologies

In [ ]:
import random
import itertools

## Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) is designed to work with a specific family of Boolean functions. Before implementing the algorithm, we need a way to generate valid test cases from that family.

### What is a Boolean Function?

A **Boolean function** maps a tuple of Boolean inputs to a single Boolean output. Here we work with functions that take **four Boolean arguments**, so each input is one of 2⁴ = 16 possible bit patterns — from `(False, False, False, False)` to `(True, True, True, True)`.

The total number of distinct 4-input Boolean functions is 2¹⁶ = 65,536, since each of the 16 inputs can independently map to True or False.

### Constant vs Balanced

The Deutsch–Jozsa algorithm is guaranteed to receive a function that is **either constant or balanced** — no other types are passed to it:

- **Constant:** The function returns the same output for *every* input. There are only **2** constant functions: one that always returns `False` and one that always returns `True`.
- **Balanced:** The function returns `True` for *exactly half* of all inputs (8 out of 16) and `False` for the other half. The number of balanced functions is C(16, 8) = **12,870**.

This distinction is the core of the [Deutsch–Jozsa problem](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa): given a black-box function promised to be one or the other, determine which type it is with as few queries as possible.

### The Generator Function

`random_constant_balanced()` generates a random function from this family. It randomly selects whether to produce a constant or balanced function, then constructs and returns a callable that implements it.

In [ ]:
def random_constant_balanced():
    """Return a randomly chosen constant or balanced Boolean function.

    The returned function accepts four Boolean arguments and returns a
    single Boolean value. It is guaranteed to be either:
      - Constant: returns the same value for all 16 possible inputs, or
      - Balanced: returns True for exactly 8 of the 16 possible inputs.

    Returns
    -------
    function
        A callable f(x1, x2, x3, x4) -> bool.
    """
    all_inputs = list(itertools.product([False, True], repeat=4))

    if random.random() < 0.5:
        value = random.choice([True, False])
        mapping = {inp: value for inp in all_inputs}

    else:
        true_inputs = set(map(tuple, random.sample(all_inputs, 8)))
        mapping = {inp: (inp in true_inputs) for inp in all_inputs}

    def f(x1, x2, x3, x4):
        """Look up the output for the given four Boolean inputs."""
        return mapping[(x1, x2, x3, x4)]

    return f

The function uses [`itertools.product`](https://docs.python.org/3/library/itertools.html#itertools.product) to enumerate all 16 input combinations without hardcoding them. Passing `repeat=4` generates every ordered 4-tuple of `False` and `True` values — equivalent to counting in 4-bit binary but using Python booleans directly.

The balanced case uses [`random.sample`](https://docs.python.org/3/library/random.html#random.sample) to choose 8 inputs without replacement, then stores them in a Python `set` so that membership checks inside the returned function run in O(1) time rather than O(n).

The function returns a **closure** — an inner function `f` that captures the `mapping` dictionary from the enclosing scope. Each call to `random_constant_balanced()` produces a fresh, independent function object.

### Correctness Verification

To confirm that `random_constant_balanced` never produces an invalid function, we generate a large sample and verify that every result is either perfectly constant or perfectly balanced.

In [ ]:
# Generate and inspect 5 random functions.
# For each function, count how many of the 16 inputs return True.
# A count of 0 or 16 confirms constant; a count of 8 confirms balanced.
print("Generating 5 random functions:\n")
print(f"{'Function':<12} {'True count':>12}  {'Type'}")
print("-" * 40)

all_inputs = list(itertools.product([False, True], repeat=4))

for i in range(5):
    f = random_constant_balanced()

    # Evaluate f on all 16 inputs and count True outputs.
    true_count = sum(f(*inp) for inp in all_inputs)

    if true_count == 0:
        func_type = "Constant (always False)"
    elif true_count == 16:
        func_type = "Constant (always True)"
    elif true_count == 8:
        func_type = "Balanced"
    else:
        func_type = f"UNEXPECTED ({true_count} True outputs)"

    print(f"Function {i+1:<3}  {true_count:>8}/16   {func_type}")

In [ ]:
# Verify correctness over a large sample.
# Every generated function must have a True count of exactly 0, 8, or 16.
trials = 1_000
invalid = 0

for _ in range(trials):
    f = random_constant_balanced()
    true_count = sum(f(*inp) for inp in all_inputs)

    # Valid True counts are 0 (const False), 16 (const True), or 8 (balanced).
    if true_count not in (0, 8, 16):
        invalid += 1

print(f"Tested {trials} generated functions.")
print(f"Invalid (neither constant nor balanced): {invalid}")
print("All functions valid." if invalid == 0 else "ERROR: invalid functions detected.")

## Problem 2: Classical Testing for Function Type

## Problem 3: Quantum Oracles

## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm